In [ ]:
# Make sure you add scikit-learn, statsmodels, and matplotlib to the project

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import streamlit as st

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import statsmodels.api as sm
import statsmodels.formula.api as smf

# Display settings
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# Lab 6: Agricultural Productivity

In this lab, we’ll be focused on feature engineering: dealing with categorical variables, missing values, and messy data. 
We’ve been working with Bayer Crop Science to understand agricultural productivity (aka crop yield) in various test fields. Unfortunately, the data was collected by high school students and isn’t the best in terms of quality and consistency. Let’s take a look and see what we can to clean up what we’ve got and still get some useful results from a multiple regression model.
Our data set is in the CSV file agri_productivity.csv.

We're giong to do the following activities:
1. Exploratory Data Analysis
2. Data Cleansing
3. Address Skew in Continuous Variables
4. Handle Missing Data
5. Dummy Coding
6. Build and Compare Models
7. Interpret Coefficients

## 1 - Exploratory Data Analysis

Take a look at the data and come with an assessment and a plan for each variable in preparing for a regression model. Show your work and then provide a markdown cell with your final conclusion for each variable.

In [ ]:
ag = pd.read_csv('agri_productivity.csv')
ag.head()

In [ ]:
ag.describe()

In [ ]:
ag.isna().sum()

### Null Handling

Looks like we're going to need to either throw out **Irrigation Method** all together or find a way to fill that. We'll look at the values in a minute and decide what to do.

With **Pesticide Use**, we may be able to fill that or just ignore those records. It's only 15 out of 300.

In [ ]:
# Simple function to plot a categorical histogram
def hist(df, cat, t):
    chart = alt.Chart(ag).mark_bar().encode(
        x = alt.X(cat, type=t, bin=(t=='quantitative')),
        y = "count()"
    )

    return chart

# Create charts for each categorical variable and display in a row
charts = [hist(ag, c, 'nominal') for c in ['Crop_Type','Irrigation_Method','Soil_Quality']]
st.altair_chart(alt.hconcat(*charts))

### Categorical Variables

**Crop Type** (nominal) and **Soil Quality** (ordinal) look fine to use.

**Irrigation Method** has LOTS of missing data and some other messy inputs. Makes you wonder how they captured this information.  Probably, we're going to need to make "null" into a kind of "unknown" value. Or maybe we just ignore that column entirely.

## 2 - Data Cleansing

Look at any categorical variables and clean up or standardize the values you find.

We've decided we need to do the following data cleansing steps:
1. Standardize **Irrigation Method** format and replace NA with "Unknown"

_Other activities like dummy coding and transformation will happen in later steps_

In [ ]:
# Strip Spaces
# Title Case (initial caps)

ag['clean_Irrigation_Method'] = ag['Irrigation_Method'].str.strip().str.title()

st.altair_chart(hist(ag, 'clean_Irrigation_Method', t='nominal'))

## 3 - Address Skew in Continuous Variables

Create histograms for any continuous variables and use a transformation to address any skew they may have.

In [ ]:
charts = [hist(ag, c, 'quantitative') for c in [
    'Fertilizer_Amount_kg_per_acre',
    'Rainfall_inches',
    'Pesticide_Use_liters_per_acre',
    'Yield_tons_per_acre']]

st.altair_chart(alt.vconcat(*charts))

### Continuous Variables

Looks like **Yield** and **Pesticide Use** are both normally distributed.

**Fertilizer** and **Rainfall** are right-skewed, however. We'll need to transform those with either a log or sqrt transformation. I'm thinking **log** for **Fertilizer** and **sqrt** for **Rainfall**

In [ ]:
# Log transform (using log1p incase there are 0s) for Fertilizer
ag['log_Fertilizer_Amount_kg_per_acre'] = np.log1p(ag['Fertilizer_Amount_kg_per_acre'])

# Sqrt transform for Rainfall
ag['sqrt_Rainfall_inches'] = np.sqrt(ag['Rainfall_inches'])

# Show plots on the transformed variables
charts = [hist(ag, c, 'quantitative') for c in [
    'log_Fertilizer_Amount_kg_per_acre',
    'sqrt_Rainfall_inches']]

st.altair_chart(alt.hconcat(*charts))

These definitely look more normally distributed.

## 4 - Handle Missing Data

Create a baseline dataframe by simply dropping any records with missing values. How many records do you have?
Evaluate each of the fields with missing data and determine how to best fill those missing values. Mode, Mean, MICE, Imputation based on other fields


We have missing values:
1. **Irrigation Method** - Let's just replace these with 'Unknown' and try that approach versus throwing out the variable
2. **Pesticide Use** - Let's use mean and iterative imputer on pesticide and compare at the end

In [ ]:
ag['imp_Irrigation_Method'] = ag['clean_Irrigation_Method'].fillna('Unknown')

st.altair_chart(hist(ag, 'imp_Irrigation_Method', t='nominal'))

In [ ]:
ag['mean_Pesticide_Use_liters_per_acre'] = ag['Pesticide_Use_liters_per_acre'].fillna(ag['Pesticide_Use_liters_per_acre'].mean())

In [ ]:
# The iterative imputer uses correlation with other columns to try to fill missing values least awkwardly
# https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html

cols = ["Pesticide_Use_liters_per_acre", "Fertilizer_Amount_kg_per_acre", "Rainfall_inches", "Yield_tons_per_acre"]

# random_state=42 to have a repeatability, any integer is fine
# sample_posterior=True because we're doing multiple imputations
# max_iter=100 because that's what I wanted... in reality, some trial and error is necessary
imputer = IterativeImputer(random_state=42, sample_posterior=True, max_iter=100)

# Create a copy of the columns we need because the imputer fills in place
imp_ag = ag[cols].copy()
imp_ag[cols] = imputer.fit_transform(imp_ag[cols])


# Copy the imputed pesticide use back to our original df
ag['imp_Pesticide_Use_liters_per_acre'] = imp_ag['Pesticide_Use_liters_per_acre']


# Show some examples where we imputed values
ag.loc[
    ag['Pesticide_Use_liters_per_acre'].isnull(),
    ['imp_Pesticide_Use_liters_per_acre','Pesticide_Use_liters_per_acre']].head()

## 5 - Dummy Coding

Use dummy encoding with each categorical variable. Make sure you can explain what to the coefficients of a dummy-coded variable means before going any further.

* Dummy code **Crop_Type** -- drop "Corn" as our reference since it was the mode
* Dummy code **Irrigation_Method** -- drop "None" as our reference
* Ordinal code **Soil Quality** as 1,2,3

In [ ]:
# prefix give us the word "Crop" in front of each dummy column name
# drop "Corn"
crop_dummies = pd.get_dummies(ag['Crop_Type'], prefix='Crop').drop(columns=['Crop_Corn'])
crop_dummies.head()

In [ ]:
# prefix with "IM"
# drop "None"
im_dummies = pd.get_dummies(ag['imp_Irrigation_Method'], prefix='IM').drop(columns=['IM_None'])
im_dummies.head()

In [ ]:
ag['ord_Soil_Quality'] = ag['Soil_Quality'].map({
    'Poor': 1,
    'Fair': 2,
    'Good': 3
})

hist(ag, 'ord_Soil_Quality', 'nominal')

In [ ]:
# Package all the dummies together into one final data frame
ag_model = pd.concat([ag, crop_dummies, im_dummies], axis=1)
ag_model.head(25)

## 6 - Build and Compare Models

Build models using the baseline data and using imputed and transformed variables. Compare the models.

1. Model with clean columns but no transformations
2. Model with basic imputation and transformations
3. Model with iterative imputation and transformations

In [ ]:
formula_1 = """
Yield_tons_per_acre ~ 
    ord_Soil_Quality + 
    Fertilizer_Amount_kg_per_acre + 
    Rainfall_inches + 
    Pesticide_Use_liters_per_acre + 
    Crop_Soybeans + Crop_Wheat + 
    IM_Drip + IM_Sprinkler + IM_Unknown"""

model_1 = smf.ols(formula=formula_1, data=ag_model).fit()
print(model_1.summary())

In [ ]:
formula_2 = """
Yield_tons_per_acre ~ 
    ord_Soil_Quality + 
    log_Fertilizer_Amount_kg_per_acre + 
    sqrt_Rainfall_inches + 
    mean_Pesticide_Use_liters_per_acre + 
    Crop_Soybeans + Crop_Wheat + 
    IM_Drip + IM_Sprinkler + IM_Unknown"""

model_2 = smf.ols(formula=formula_2, data=ag_model).fit()
print(model_2.summary())

In [ ]:
formula_3 = """
Yield_tons_per_acre ~ 
    ord_Soil_Quality + 
    log_Fertilizer_Amount_kg_per_acre + 
    sqrt_Rainfall_inches + 
    imp_Pesticide_Use_liters_per_acre + 
    Crop_Soybeans + Crop_Wheat + 
    IM_Drip + IM_Sprinkler + IM_Unknown"""

model_3 = smf.ols(formula=formula_3, data=ag_model).fit()
print(model_3.summary())

### Model Interpretation

```
Model                                    Adj R2
Model 1 (Cleansed, No Transformations)    0.337
Model 2 (Simple Cleans, Transformed)      0.353
Model 3 (Imputations, Transformed)        0.354
```

We see improvement in the R2 for each iteration. 

We've got strong p-values for Soil Qualtiy, Fertilizer Amount, and Rainfall in all models. So, it's good to have consistency.



## 7 - Interpret Coefficients

Explain what the various coefficients mean. Remember to adjust your interpretation based on any transformations.

In [ ]:
coefs = pd.DataFrame({
    'Model 1 (clean, no transform)': model_1.params,
    'Model 2 (mean, transformed)': model_2.params,
    'Model 3 (impute, transformed)': model_3.params
})

coefs

While only Soil Quality, Fertilizer Amount, and Rainfall have significance, we can provide the following interpretation of the coefficients. Yield in tons per acre increases by 1 ton based on the following changes based on our best model (model 3):

* If the crop is Soybeans rather than Corn: -0.1404
* If the crop is What rather than Corn: -0.1294
* ... That is, **Corn may provide the highest yield of all the crop types**
* If Drip irrigation is definitely being used rather than nothing: 0.2333
* If Sprinkler irrigation is definitely being used rather than nothing: 0.0668
* If irrigation is Unknown: -0.2023
* ... That is, **Drip irrigation provides the highest boost to crop yeild**
* **Pesticide use delivers 0.0673 to 0.0754 more yield for each additional liter of pesticide per acre**
* **Each step up in Soil Quality delivers a 0.2553 to 0.2639 increase in tons yeild per acre**
* An increase of **1 unit in the square root of inches of Rainfall increases yield by 0.1486 tons per acre**

**If we can only choose one thing to do with our crops, choosing Drip irrigation has the greatest impact on yeild!**